In [1]:
import pandas as pd
import numpy as np
import warnings
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

In [10]:
sales = pd.read_csv("data/original_sales_data.csv")

In [15]:
sales.head()

,order_number,order_date,sku_id,warehouse_id,customer_type,order_quantity,unit_sale_price,revenue,adjusted_order_quantity
0,SO - 018900,2021-01-01,3551CA,GUT930,Export,105.00,7.07,742,105.00
1,SO - 018901,2021-01-01,3079BA,AXW291,Wholesale,128.00,134.50,20310,128.00
2,SO - 018902,2021-01-01,3250CA,AXW291,Distributor,185.33,34.75,10426,185.33
3,SO - 018903,2021-01-01,1161AA,GUT930,Wholesale,167.00,136.59,6830,167.00
4,SO - 018904,2021-01-01,3512AA,GUT930,Distributor,450.00,0.10,103,450.00


In [16]:
sales['order_date'] = pd.to_datetime(sales['order_date'])
sales = sales.loc[:, ~sales.columns.str.startswith('Unnamed')]
sales=sales[sales['order_date']<"2022-12-31"]
sales = sales.set_index('order_date')

In [11]:
# Clean the dataset by dropping unnecessary columns
sales_data_cleaned = sales.drop(columns=["Unnamed: 8", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "Unnamed: 12"])

# Convert 'order_date' to datetime
sales_data_cleaned['order_date'] = pd.to_datetime(sales_data_cleaned['order_date'], format='%d-%b-%y')

# Prepare the data for SKU-level predictions
# Aggregate sales data at the daily level for each SKU
sku_daily_sales = sales_data_cleaned.groupby(['order_date', 'sku_id'])['order_quantity'].sum().reset_index()

# Sort data by SKU and date
sku_daily_sales = sku_daily_sales.sort_values(by=['sku_id', 'order_date'])

# Feature Engineering: Create lag and rolling features
sku_daily_sales['lag_1'] = sku_daily_sales.groupby('sku_id')['order_quantity'].shift(1)
sku_daily_sales['lag_7'] = sku_daily_sales.groupby('sku_id')['order_quantity'].shift(7)
sku_daily_sales['rolling_avg_7_days'] = sku_daily_sales.groupby('sku_id')['order_quantity'].transform(lambda x: x.rolling(window=7).mean())
sku_daily_sales['cumulative_sum'] = sku_daily_sales.groupby('sku_id')['order_quantity'].cumsum()

# Add time-based features
sku_daily_sales['year'] = sku_daily_sales['order_date'].dt.year
sku_daily_sales['month'] = sku_daily_sales['order_date'].dt.month
sku_daily_sales['day_of_week'] = sku_daily_sales['order_date'].dt.dayofweek

# Drop rows with NaN values created by lagging and rolling features
sku_daily_sales = sku_daily_sales.dropna()

# Display the processed dataset
sku_daily_sales.head()


,order_date,sku_id,order_quantity,lag_1,lag_7,rolling_avg_7_days,cumulative_sum,year,month,day_of_week
520,2021-01-15,1009AA,393.00,905.00,915.00,506.00,4457.00,2021,1,4
592,2021-01-17,1009AA,70.00,393.00,140.00,496.00,4527.00,2021,1,6
712,2021-01-20,1009AA,60.00,70.00,1220.00,330.29,4587.00,2021,1,2
751,2021-01-21,1009AA,100.00,60.00,454.00,279.71,4687.00,2021,1,3
788,2021-01-22,1009AA,300.00,100.00,150.00,301.14,4987.00,2021,1,4


In [ ]:
# Features and target variable
features = ['lag_1', 'lag_7', 'rolling_avg_7_days', 'cumulative_sum', 'year', 'month', 'day_of_week']
target = 'order_quantity'

# Split data temporally: Use last 20% of dates as the test set
split_date = sku_daily_sales['order_date'].quantile(0.8)
train_data = sku_daily_sales[sku_daily_sales['order_date'] <= split_date]
test_data = sku_daily_sales[sku_daily_sales['order_date'] > split_date]

# Separate features and target for train and test sets
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

In [23]:
from sklearn.ensemble import RandomForestRegressor

# Train a Random Forest model
rf_model = RandomForestRegressor(random_state=42, n_estimators=200)
rf_model.fit(X_train, y_train)

# Predict on the test set
rf_predictions = rf_model.predict(X_test)

# Evaluate the model
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_rmse = np.sqrt(rf_mse)
rf_r2 = r2_score(y_test, rf_predictions)

rf_mse, rf_rmse, rf_r2


(np.float64(2816704.6888483097),
 np.float64(1678.3041109549574),
 0.6055110391013181)

In [26]:
from xgboost import XGBRegressor

# Train an XGBoost model
xgb_model = XGBRegressor(random_state=42, n_estimators=100, learning_rate=0.03)
xgb_model.fit(X_train, y_train)

# Predict on the test set
xgb_predictions = xgb_model.predict(X_test)

# Evaluate the XGBoost model
xgb_mse = mean_squared_error(y_test, xgb_predictions)
xgb_rmse = np.sqrt(xgb_mse)
xgb_r2 = r2_score(y_test, xgb_predictions)

xgb_mse, xgb_rmse, xgb_r2


(np.float64(2821137.0127188023),
 np.float64(1679.6240688674363),
 0.6048902772426257)

In [18]:
def create_time_series_features(df: pd.DataFrame):
    features = {
        'quarter': df.index.quarter,
        'day_of_week': df.index.dayofweek,
        'date_and_month': df.index.strftime('%m %b'),
        'month': df.index.month,
        'year': df.index.year,
        'day_of_month': df.index.day,
        'week_of_year': df.index.isocalendar().week
    }

    for feature_name, feature_values in features.items():
        df[feature_name] = feature_values

    return df

sales = create_time_series_features(sales)

In [19]:
split_date = '2022-08-31'

train = sales.loc[sales.index < split_date]
test = sales.loc[sales.index >= split_date]

Filter the attributes that are needed for training and testing:

In [21]:
training_filtered_sales_data = train[['sku_id','customer_type','day_of_month','day_of_week','month','quarter','year','week_of_year','order_quantity']]
testing_filtered_sales_data = test[['sku_id','customer_type','day_of_month','day_of_week','month','quarter','year','week_of_year','order_quantity']]

In [22]:
def oneHotEncoding(df, index_column):
    df=df.set_index(index_column)
    
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

    encoder = OneHotEncoder(sparse_output=False)
    one_hot_encoded_categorical = encoder.fit_transform(df[categorical_cols])
    one_hot_df = pd.DataFrame(one_hot_encoded_categorical, columns=encoder.get_feature_names_out(categorical_cols))
    
    one_hot_encoded = pd.concat([df.reset_index(),one_hot_df], axis=1)
    one_hot_encoded = one_hot_encoded.drop(categorical_cols, axis=1)
    df = one_hot_encoded.set_index(index_column)

    return df

In [23]:
training_sales_encoded = oneHotEncoding(training_filtered_sales_data, 'sku_id')
testing_sales_encoded = oneHotEncoding(testing_filtered_sales_data, 'sku_id')

In [24]:
features = training_sales_encoded.columns.values.tolist()
features.remove('order_quantity')
target = 'order_quantity'

In [25]:
X_train = training_sales_encoded[features]
y_train = training_sales_encoded[target]

X_test = testing_sales_encoded[features]
y_test = testing_sales_encoded[target]

In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the Random Forest Regressor with hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Best model from grid search
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate accuracy metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 2187917.95052384
R^2 Score: -0.09932309221989821
